In [ ]:
# SENTINEL2_GETDATA
# Get sSentinel2 data products based on an area of interest, cloudcover and date interval
# The earliest or the latest within a set interval, depending on your preferences
# (see below)
# Result is saved to a designated directory
# RTS, Feb 2021
#-------------------------------------------------------------------------------
# Preparations:
# 1
# Create an account with Copernicus Open Access Hub (https://scihub.copernicus.eu/dhus)
# You will need a login and pswd to access sentinel2 files
# 2
# Generate a geojson file for area of interest (https://geojson.io/)
# Save the file as somearea.geojson and place a copy on into your data directory (see below)

# ESA quotas:
# https://forum.step.esa.int/t/esa-copernicus-data-access-long-term-archive-and-its-drawbacks/15394/14
# Please note that the maximum number of products that a single user can request on SciHub is 1 every 30 minutes. 
# An additional quota limit is applied to users of the APIHub of maximum 20 products every 12 hours. 

In [ ]:
import sys
#clear all variables (from last session)
sys.modules[__name__].__dict__.clear()
import os, sys, time, random, itertools, shutil

from google.colab import drive
drive.mount('/content/drive')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Mounted at /content/drive


In [ ]:
#all the imports here
!pip install sentinelsat --upgrade
print('\nFINISHED installing sentinelsat....')

In [ ]:
# variables specific to your CoLabsetup ----------------------------------------
start = '/content/drive/MyDrive/your directory/'
datapath = start + 'data/'
searchpath = datapath + 'sentinel/'
archivepath = datapath + 'archive/'

# make the directories if they do not exist ------------------------------------
all_dirs = [datapath, searchpath, archivepath]

for dir in all_dirs:
  if (not os.path.exists(dir)):
    os.makedirs(dir)
    print('Created this directory: ', dir)
  else:
    print('This directory already exists: ', dir)

print('\nFINISHED setting directories....')

In [ ]:
# Sentinel sepcific paramenters, login, password and geojson file
# this setup assumes you have a valid geojson file in the .../data directory

user =   'yourlogin'
pswd =   'yourpassword'

#set the max acceptable cloud cover
maxcover = 20
#set the number of days or months to search into the past (from the present date)
ndays = 12
nmonths = 1

#set the desired area via the geojson file
map = 'manhattan.geojson'         #'your.geojson'
geojsonpath = datapath

print('Sentinel search parameters are set....')

In [ ]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

api = SentinelAPI(user, pswd, 'https://scihub.copernicus.eu/dhus')

# start is length of days or months to look into the past.
# the search will begin there.

#start = 'NOW-' + str(ndays) + 'DAYS'
start = 'NOW-' + str(nmonths) + 'MONTHS'

maxitems = 1
end = 'NOW'
platform = 'Sentinel-2'
product = 'S2MSI1C'
footprint = geojson_to_wkt(read_geojson(geojsonpath + map))

print('GEOJSON MAP: ', map)
print('TIMEFRAME: ', start, end)
print('MAX CLOUD COVER: ', maxcover)
print('PLATFORM, PRODUCT: ', platform, product)

try:
  products = api.query(footprint, date = (start, end), platformname = platform, producttype = product, cloudcoverpercentage = (0, maxcover))
  products_df = api.to_dataframe(products)
  # ascending=[False] > newest item; ascending=[True] > oldest item;
  products_df_sorted = products_df.sort_values(['ingestiondate'], ascending=[True])  
  # usually only 1 (large files, upto 1GB )         
  print('\nNumber of items being downloaded: ', maxitems)
  products_df_sorted = products_df_sorted.head(maxitems)                                       
  print('\nGetting this/these item/s: ', products_df_sorted)
  api.download_all(products_df_sorted.index, directory_path = searchpath)
  print('\nDownload attempt complete ...  check sentinel folder to confirm data is ok')

# catch all exceptions
except Exception as ex:
  template = "\nAn exception of type {0} occurred. This is the reported error:\n{1!r}"
  message = template.format(type(ex).__name__, ex.args)
  print (message) 
  print('\n.. Something went wrong ... see message below... other possible error sources: geojson file, ndays, nmonths and cloud cover setting ...')
